## Exploración de campos de los datos

En este paso, cargaremos los archivos `credits.csv` y `movies_metadata.csv` (o cualquier otro nombre que tengan) y exploraremos los nombres de las columnas para identificar en qué archivo se encuentra cada campo.


In [5]:
import pandas as pd
import ast

# Cargar ambos archivos para ver sus columnas
credits_df = pd.read_csv('final_data/credits_small.csv')
movies_df = pd.read_csv('data/movies_dataset.csv')  # Asegúrate de usar el nombre correcto

# Mostrar los nombres de las columnas
print(*pd.read_csv('data/credits.csv').columns, sep="\n")
print("\n")
print(*pd.read_csv('data/movies_dataset.csv').columns, sep="\n")


C:\Users\Jose Lara\AppData\Local\Temp\ipykernel_17216\3237471977.py:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df = pd.read_csv('data/movies_dataset.csv')  # Asegúrate de usar el nombre correcto


cast
crew
id


adult
belongs_to_collection
budget
genres
homepage
id
imdb_id
original_language
original_title
overview
popularity
poster_path
production_companies
production_countries
release_date
revenue
runtime
spoken_languages
status
tagline
title
video
vote_average
vote_count


C:\Users\Jose Lara\AppData\Local\Temp\ipykernel_17216\3237471977.py:11: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  print(*pd.read_csv('data/movies_dataset.csv').columns, sep="\n")


### Primera Transformación

A. Desanidar Columnas con Estructura de Diccionario o Lista

Paso 1: Desanidar belongs_to_collection

In [6]:
# Desanidar `belongs_to_collection` en nuevas columnas
movies_df['belongs_to_collection_id'] = movies_df['belongs_to_collection'].apply(
    lambda x: ast.literal_eval(x).get('id') if pd.notna(x) and isinstance(x, str) and x.startswith('{') else None
)
movies_df['belongs_to_collection_name'] = movies_df['belongs_to_collection'].apply(
    lambda x: ast.literal_eval(x).get('name') if pd.notna(x) and isinstance(x, str) and x.startswith('{') else None
)
movies_df['belongs_to_collection_poster_path'] = movies_df['belongs_to_collection'].apply(
    lambda x: ast.literal_eval(x).get('poster_path') if pd.notna(x) and isinstance(x, str) and x.startswith('{') else None
)
movies_df['belongs_to_collection_backdrop_path'] = movies_df['belongs_to_collection'].apply(
    lambda x: ast.literal_eval(x).get('backdrop_path') if pd.notna(x) and isinstance(x, str) and x.startswith('{') else None
)

# Eliminar la columna original
movies_df.drop(columns=['belongs_to_collection'], inplace=True)

Paso 2: Desanidar genres

In [7]:
# Extraer nombres e ids de géneros en listas separadas
movies_df['genres_ids'] = movies_df['genres'].apply(
    lambda x: [i['id'] for i in ast.literal_eval(x) if isinstance(i, dict)] 
    if isinstance(x, str) and x.startswith('[') 
    else []
)
movies_df['genres_names'] = movies_df['genres'].apply(
    lambda x: [i['name'] for i in ast.literal_eval(x) if isinstance(i, dict)] 
    if isinstance(x, str) and x.startswith('[') 
    else []
)

# Eliminar la columna original
movies_df.drop(columns=['genres'], inplace=True)

Paso 3: Desanidar production_companies

In [8]:
# Extraer nombres e ids de compañías en listas separadas
movies_df['production_companies_ids'] = movies_df['production_companies'].apply(
    lambda x: [i['id'] for i in ast.literal_eval(x) if isinstance(i, dict)] 
    if isinstance(x, str) and x.startswith('[') 
    else []
)
movies_df['production_companies_names'] = movies_df['production_companies'].apply(
    lambda x: [i['name'] for i in ast.literal_eval(x) if isinstance(i, dict)] 
    if isinstance(x, str) and x.startswith('[') 
    else []
)

# Eliminar la columna original
movies_df.drop(columns=['production_companies'], inplace=True)

Paso 4: Desanidar production_countries

In [9]:
# Extraer códigos ISO y nombres de países en listas separadas
movies_df['production_countries_iso'] = movies_df['production_countries'].apply(
    lambda x: [i['iso_3166_1'] for i in ast.literal_eval(x) if isinstance(i, dict)] 
    if isinstance(x, str) and x.startswith('[') 
    else []
)
movies_df['production_countries_names'] = movies_df['production_countries'].apply(
    lambda x: [i['name'] for i in ast.literal_eval(x) if isinstance(i, dict)] 
    if isinstance(x, str) and x.startswith('[') 
    else []
)

# Eliminar la columna original
movies_df.drop(columns=['production_countries'], inplace=True)

Paso 5: Verificación de Resultados

In [10]:
print(movies_df.columns)

Index(['adult', 'budget', 'homepage', 'id', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path',
       'release_date', 'revenue', 'runtime', 'spoken_languages', 'status',
       'tagline', 'title', 'video', 'vote_average', 'vote_count',
       'belongs_to_collection_id', 'belongs_to_collection_name',
       'belongs_to_collection_poster_path',
       'belongs_to_collection_backdrop_path', 'genres_ids', 'genres_names',
       'production_companies_ids', 'production_companies_names',
       'production_countries_iso', 'production_countries_names'],
      dtype='object')


### Segunda Transformación

Rellenar valores nulos en los campos `revenue` y `budget`

En este paso, se reemplazarán los valores nulos en los campos `revenue` y `budget` con el número `0`. Esto garantiza que estos campos numéricos no tengan valores faltantes, facilitando su uso en cálculos y análisis posteriores.

In [11]:
# Rellenar valores nulos en `revenue` y `budget` con 0
movies_df['revenue'].fillna(0, inplace=True)
movies_df['budget'].fillna(0, inplace=True)

C:\Users\Jose Lara\AppData\Local\Temp\ipykernel_17216\725850694.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  movies_df['revenue'].fillna(0, inplace=True)
C:\Users\Jose Lara\AppData\Local\Temp\ipykernel_17216\725850694.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For e

### Tercera Transformación 

Eliminar valores nulos en el campo `release_date`

En este paso, se eliminan todas las filas que tienen un valor nulo en el campo `release_date`. Esto asegura que todas las películas en el conjunto de datos tengan una fecha de lanzamiento, lo cual es importante para análisis basados en fechas.

In [12]:
# Eliminar filas con valores nulos en el campo `release_date`
movies_df.dropna(subset=['release_date'], inplace=True)

## Cuarta Transformación

Fechas y extraer año de lanzamiento

En este paso, las fechas en el campo `release_date` se convierten al formato estándar `AAAA-mm-dd` para una mayor consistencia. Además, se crea una nueva columna llamada `release_year`, que extrae el año de la fecha de estreno, facilitando los análisis que requieran información sobre el año de lanzamiento.


In [13]:
# Convertir `release_date` al formato AAAA-mm-dd
movies_df['release_date'] = pd.to_datetime(movies_df['release_date'], errors='coerce').dt.strftime('%Y-%m-%d')

# Crear la columna `release_year` extrayendo el año de `release_date`
movies_df['release_year'] = pd.to_datetime(movies_df['release_date'], errors='coerce').dt.year

In [14]:
# Ver los primeros valores de las columnas 'release_date', 'release_year' para verificarlos
movies_df[['release_date', 'release_year']]

,release_date,release_year
0,1995-10-30,1995.0
1,1995-12-15,1995.0
2,1995-12-22,1995.0
3,1995-12-22,1995.0
4,1995-02-10,1995.0
...,...,...
45460,1991-05-13,1991.0
45462,2011-11-17,2011.0
45463,2003-08-01,2003.0
45464,1917-10-21,1917.0


## Quinta Transformación

Calcular el Retorno de Inversión (ROI)

En este paso, se crea una nueva columna llamada `return`, que representa el retorno de inversión de cada película. Este valor se calcula dividiendo el campo `revenue` entre `budget`. Si `budget` es igual a 0, el valor 


In [15]:
# Convertir las columnas `revenue` y `budget` a tipo numérico, reemplazando valores no convertibles por 0
movies_df['revenue'] = pd.to_numeric(movies_df['revenue'], errors='coerce').fillna(0)
movies_df['budget'] = pd.to_numeric(movies_df['budget'], errors='coerce').fillna(0)

# Crear la columna `return` calculando el retorno de inversión (revenue / budget)
movies_df['return'] = movies_df.apply(lambda row: row['revenue'] / row['budget'] if row['budget'] != 0 else 0, axis=1)

In [16]:
# Ver los primeros valores de las columnas `revenue`, `budget` y `return` para verificarlos
movies_df[['revenue', 'budget', 'return']]

,revenue,budget,return
0,373554033.0,30000000.0,12.451801
1,262797249.0,65000000.0,4.043035
2,0.0,0.0,0.000000
3,81452156.0,16000000.0,5.090760
4,76578911.0,0.0,0.000000
...,...,...,...
45460,0.0,0.0,0.000000
45462,0.0,0.0,0.000000
45463,0.0,0.0,0.000000
45464,0.0,0.0,0.000000


## Sexta Transformación

Eliminar columnas no utilizadas

En este paso, se eliminan las columnas que no serán utilizadas en el análisis: `video`, `imdb_id`, `adult`, `original_title`, `poster_path`, y `homepage`. Esto simplifica el conjunto de datos, manteniendo solo las columnas relevantes para el análisis.


In [17]:
# Eliminar las columnas que no serán utilizadas
movies_df.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'], inplace=True)

In [18]:
# Verificación de los campos restantes
print(movies_df.columns)

Index(['budget', 'id', 'original_language', 'overview', 'popularity',
       'release_date', 'revenue', 'runtime', 'spoken_languages', 'status',
       'tagline', 'title', 'vote_average', 'vote_count',
       'belongs_to_collection_id', 'belongs_to_collection_name',
       'belongs_to_collection_poster_path',
       'belongs_to_collection_backdrop_path', 'genres_ids', 'genres_names',
       'production_companies_ids', 'production_companies_names',
       'production_countries_iso', 'production_countries_names',
       'release_year', 'return'],
      dtype='object')


Creando un nuevo CSV para los datos ya transformados

In [19]:
# Guardar la última versión de movies_df en la carpeta final_data
movies_df.to_csv('final_data/final_movies.csv', index=False)